In [1]:
%reload_ext autoreload
%autoreload 2
import os
os.chdir('../')
import pandas as pd
import numpy as np
from evaluator.gpt_evaluator import FinancialDataProcessor
import time

/home/ubuntu/miniforge3/envs/multimodal/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
def wait_for_completion(job_id, processor, poll_interval=100):
    status = processor.check_status(job_id)
    while status.status not in ["completed", "failed"]:
        print(f"Current status: {status}. Waiting for {poll_interval} seconds...")
        time.sleep(poll_interval)
        status = processor.check_status(job_id)
    return status.status

# Financial Data

In [3]:
financial_data_processor = FinancialDataProcessor()

results_dir = f"/home/ubuntu/multimodal/Data/financial/"
data = pd.read_csv(f"/home/ubuntu/multimodal/Data/financial/CRM.csv")

jsonl_path = os.path.join(results_dir, "batch.jsonl")
output_path = os.path.join(results_dir, "data.txt")

# batch_object_id = financial_data_processor.create_and_run_batch_job(data, jsonl_path, input_text_column="text")

# job_status = wait_for_completion(batch_object_id, financial_data_processor)

# if job_status == "completed":
#     print("Batch job completed successfully!")
#     semantic_outputs = financial_data_processor.check_status_and_parse(batch_object_id , output_path)

In [4]:
semantic_outputs = financial_data_processor.check_status_and_parse("batch_cOPRrrflm4aTTHJN2cG45ZGb" , output_path)

In [15]:
print(data.iloc[7]['timestamp'], semantic_outputs[7])

2022-03-29 **HP Inc. (HPQ) Acquires Poly for $3.3 Billion**

- **Acquisition Details**: HP Inc. announced a definitive agreement to acquire Poly, a provider of workplace collaboration solutions, for $3.3 billion. HP will pay $40 per share for Poly, which represents a 53% premium over the last closing price.
- **Market Reaction**: Following the announcement, Poly's shares surged by 52.6%, while HP's shares dropped by 2.7%.
- **Transaction Value**: Including Poly’s debt, the acquisition is valued at $3.3 billion and is expected to close by the end of 2022, pending shareholder approval.
  
**Strategic Importance**:
- This acquisition aims to enhance HP’s peripheral business and capitalize on the increasing demand for hybrid work solutions post-pandemic.
- The peripheral market is currently a $110 billion opportunity, growing at 9% annually, while workplace collaboration solutions represent a $120 billion market growing at 8% annually.
- HP forecasts that the acquisition will be immediatel

In [14]:
print(data.iloc[6]['timestamp'], semantic_outputs[6])


2022-03-29 **Key Stock-Related Details:**

- **Company:** Weimob Inc. (2013.HK)
- **Financial Performance (2021):**
  - Adjusted net loss: 566 million yuan (reversed from a profit of 107.5 million yuan in 2020).
  - Revenue growth: 36.4%, totaling 2.7 billion yuan.
  - Costs significantly outpaced revenue growth, particularly selling/distribution expenses increased by 87%.
  
- **Investor Insights:**
  - Weimob's stock jumped 18% following their earnings report despite the losses.
  - The stock has lost more than two-thirds of its value over the last 52 weeks.
  - Currently trades at a price-to-sales (P/S) ratio of 3.4, down from over 7.8 a year prior. 

- **Comparison with Peers:**
  - U.S. competitor Salesforce (CRM) has a P/S ratio of 7.8.
  - Domestic rival Baozun (BZUN) has a notably low P/S ratio of 0.4.

- **Investor Activity:**
  - Big-name institutional investors are showing early signs of returning after previously selling off.
  - Notable stake increases:
    - JPMorgan: 8.3